# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
df = pd.read_csv('weather.csv')
df

,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,24.2231,-107.1869,291.48,98,34,1.25,1619611139
1,19.7297,-155.0900,296.15,78,90,1.54,1619611139
2,-54.8000,-68.3000,275.15,100,40,4.63,1619610967
3,69.4865,88.3972,274.05,99,99,3.95,1619611140
4,39.0500,-27.9833,288.27,80,100,6.96,1619611140
...,...,...,...,...,...,...,...
569,55.5281,41.1053,280.95,66,98,4.06,1619611332
570,41.1239,-100.7654,281.15,93,90,6.01,1619611099
571,64.8569,-147.8028,272.15,59,75,2.10,1619611332
572,-8.4667,140.3333,299.35,85,73,5.32,1619611270


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#geo_url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={df['Lat']},{df['Long']}&key={g_key}"
#geo_url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng=-15.8631,-38.8828&key={g_key}"
#geo_data = requests.get(geo_url).json()
#print(geo_data)

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Long"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#census_data_complete = census_data_complete.dropna()
humidity_rate = df["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
good_weather = df[df['Max Temp']>297.04]
hotel_df=good_weather[good_weather['Cloudiness']<50]
hotel_df

,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
16,5.4112,100.3354,303.15,74,20,2.57,1619611024
18,-17.7333,-149.1500,298.15,83,20,1.03,1619611144
21,-23.1203,-134.9692,297.27,76,32,3.14,1619611145
58,14.3122,121.1114,302.04,74,25,2.68,1619610911
64,14.9215,-23.5087,298.15,57,40,6.69,1619611027
...,...,...,...,...,...,...,...
557,3.1667,113.0333,302.15,79,20,1.54,1619611329
558,14.7328,-6.1219,313.15,17,30,3.57,1619611329
564,-4.0625,144.0661,297.92,88,36,1.50,1619611331
565,20.7833,86.7667,303.57,63,2,8.04,1619611331


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
cities = []
hotels = []

for i, rows in hotel_df.iterrows():
    t = hotel_df['Lat'][i].astype(str)
    c = hotel_df['Long'][i].astype(str)
    target_coord = t + ',' + c
#    target_coord = "33.75, -84.388"
    target_radius = 5000
    target_type = "hotel"
#    target_search = "Chinese"

    params = {
        "location": target_coord,
#        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }


    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places_data = requests.get(base_url, params=params).json()
#    print(places_data["results"][1]["name"])
    cities.append(places_data["results"][0]["name"])
    hotels.append(places_data["results"][1]["name"])
    


In [91]:
hotel_df['City'] = cities
hotel_df['Hotel Name'] = hotels

hotel_df

C:\Users\17709\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\17709\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,City
16,5.4112,100.3354,303.15,74,20,2.57,1619611024,Cititel Penang,George Town
18,-17.7333,-149.1500,298.15,83,20,1.03,1619611144,Pension Oaoa,Tautira
21,-23.1203,-134.9692,297.27,76,32,3.14,1619611145,Pension Maro'i,Rikitea
58,14.3122,121.1114,302.04,74,25,2.68,1619610911,Erlyn’s Sta Rosa City Home,Santa Rosa
64,14.9215,-23.5087,298.15,57,40,6.69,1619611027,LT Aparthotel,Praia
...,...,...,...,...,...,...,...,...,...
557,3.1667,113.0333,302.15,79,20,1.54,1619611329,900 Boutique INN,Bintulu
558,14.7328,-6.1219,313.15,17,30,3.57,1619611329,Grande Mosquée de Sokolo,Sokolo
564,-4.0625,144.0661,297.92,88,36,1.50,1619611331,Angoram Primary School,Angoram
565,20.7833,86.7667,303.57,63,2,8.04,1619611331,State Bank ATM,Chandbali


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [101]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel: {place}" for place in hotels])

fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))